In [1]:
import numpy as np
import torch 
from torch import nn
import torch.nn.functional as F

In [2]:
with open('data/anna.txt','r') as f:
    text=f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
#tokenization

chars=tuple(set(text))
int2char=dict(enumerate(chars))
char2int= {ch: ii for ii,ch in int2char.items()}

encoded =np.array([char2int[ch] for ch in text])



In [5]:
int2char.items()

dict_items([(0, '.'), (1, '/'), (2, ':'), (3, 'A'), (4, 'p'), (5, '0'), (6, 'u'), (7, ';'), (8, '1'), (9, '`'), (10, '2'), (11, 'q'), (12, 'W'), (13, 'U'), (14, 'F'), (15, 'X'), (16, ','), (17, 'P'), (18, 'i'), (19, 'g'), (20, '-'), (21, 'L'), (22, 'B'), (23, 'f'), (24, '6'), (25, '7'), (26, 'J'), (27, '*'), (28, 'b'), (29, 'j'), (30, 'N'), (31, '_'), (32, 'E'), (33, '3'), (34, 'Z'), (35, '5'), (36, '8'), (37, '9'), (38, 'Q'), (39, 'H'), (40, 'x'), (41, 'C'), (42, 'G'), (43, '?'), (44, 'R'), (45, 'S'), (46, 'y'), (47, 'z'), (48, '!'), (49, 'm'), (50, 'V'), (51, '\n'), (52, 'e'), (53, 'I'), (54, "'"), (55, 'w'), (56, 'Y'), (57, 'K'), (58, 'h'), (59, 'a'), (60, 'T'), (61, '4'), (62, '%'), (63, '$'), (64, 'k'), (65, 'l'), (66, 'D'), (67, 'M'), (68, 't'), (69, 'o'), (70, '&'), (71, '@'), (72, 's'), (73, '"'), (74, 'c'), (75, ' '), (76, 'n'), (77, ')'), (78, 'O'), (79, 'r'), (80, 'd'), (81, 'v'), (82, '(')])

In [6]:
encoded[:100]

array([41, 58, 59,  4, 68, 52, 79, 75,  8, 51, 51, 51, 39, 59,  4,  4, 46,
       75, 23, 59, 49, 18, 65, 18, 52, 72, 75, 59, 79, 52, 75, 59, 65, 65,
       75, 59, 65, 18, 64, 52,  7, 75, 52, 81, 52, 79, 46, 75,  6, 76, 58,
       59,  4,  4, 46, 75, 23, 59, 49, 18, 65, 46, 75, 18, 72, 75,  6, 76,
       58, 59,  4,  4, 46, 75, 18, 76, 75, 18, 68, 72, 75, 69, 55, 76, 51,
       55, 59, 46,  0, 51, 51, 32, 81, 52, 79, 46, 68, 58, 18, 76])

In [7]:
char2int.items()

dict_items([('.', 0), ('/', 1), (':', 2), ('A', 3), ('p', 4), ('0', 5), ('u', 6), (';', 7), ('1', 8), ('`', 9), ('2', 10), ('q', 11), ('W', 12), ('U', 13), ('F', 14), ('X', 15), (',', 16), ('P', 17), ('i', 18), ('g', 19), ('-', 20), ('L', 21), ('B', 22), ('f', 23), ('6', 24), ('7', 25), ('J', 26), ('*', 27), ('b', 28), ('j', 29), ('N', 30), ('_', 31), ('E', 32), ('3', 33), ('Z', 34), ('5', 35), ('8', 36), ('9', 37), ('Q', 38), ('H', 39), ('x', 40), ('C', 41), ('G', 42), ('?', 43), ('R', 44), ('S', 45), ('y', 46), ('z', 47), ('!', 48), ('m', 49), ('V', 50), ('\n', 51), ('e', 52), ('I', 53), ("'", 54), ('w', 55), ('Y', 56), ('K', 57), ('h', 58), ('a', 59), ('T', 60), ('4', 61), ('%', 62), ('$', 63), ('k', 64), ('l', 65), ('D', 66), ('M', 67), ('t', 68), ('o', 69), ('&', 70), ('@', 71), ('s', 72), ('"', 73), ('c', 74), (' ', 75), ('n', 76), (')', 77), ('O', 78), ('r', 79), ('d', 80), ('v', 81), ('(', 82)])

In [8]:
# pre processing 
def one_hot_encode(arr,n_labels):
    one_hot=np.zeros((arr.size,n_labels),dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]),arr.flatten()]=1.
    one_hot=one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [9]:
#testing function
test_seq=np.array([[3,4,1,4]])
one_hot=one_hot_encode(test_seq,8)
print(test_seq)
print(one_hot)

[[3 4 1 4]]
[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]]]


In [13]:
def get_batches(arr,batch_size, seq_length):
    batch_size_total=batch_size*seq_length
    n_batches=len(arr)//batch_size_total
    
    arr=arr[:n_batches*batch_size_total]
    arr=arr.reshape((batch_size,-1))
    
    for n in range(0,arr.shape[1],seq_length):
        x=arr[:,n:n+seq_length]
        y=np.zeros_like(x)
        try:
            y[:,:-1],y[:,-1] =x[:,1:],arr[:,n+seq_length]
        except Indexerror:
            y[:,:-1],y[:,-1]=x[:,1:],arr[:,0]
        yield x,y

In [14]:
batches=get_batches(encoded, 8, 50)
x,y =next(batches)

In [15]:
print('x\n',x[:10,:10])
print('\ny\n',y[:10,:10])

x
 [[41 58 59  4 68 52 79 75  8 51]
 [72 69 76 75 68 58 59 68 75 59]
 [52 76 80 75 69 79 75 59 75 23]
 [72 75 68 58 52 75 74 58 18 52]
 [75 72 59 55 75 58 52 79 75 68]
 [74  6 72 72 18 69 76 75 59 76]
 [75  3 76 76 59 75 58 59 80 75]
 [78 28 65 69 76 72 64 46  0 75]]

y
 [[58 59  4 68 52 79 75  8 51 51]
 [69 76 75 68 58 59 68 75 59 68]
 [76 80 75 69 79 75 59 75 23 69]
 [75 68 58 52 75 74 58 18 52 23]
 [72 59 55 75 58 52 79 75 68 52]
 [ 6 72 72 18 69 76 75 59 76 80]
 [ 3 76 76 59 75 58 59 80 75 72]
 [28 65 69 76 72 64 46  0 75 73]]


In [20]:
# defining LSTM layer and class
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob=drop_prob
        self.n_layers=n_layers
        self.n_hidden=n_hidden
        self.lr=lr
        
        self.chars=tokens
        self.int2char=dict(enumerate(self.chars))
        self.char2int={ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm==nn.LSTM(input_size, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout=nn.Dropout(drop_prob)
        seld.fc=nn.Linear(n_hidden, len(self.chars))
        
    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out= self.dropout(r_output)
        out= out.contiguous().view(-1, self.n_hidden)
        out=self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight=next(self.parameters()).data
        
        hidden= (weight.new(self.n_layers, batch_size, self.n_hidden).zero(),
                 weight.new(self.n_layers, batch_size, self.n_hidden).zero())
        
        return hidden